## Problem Statement
The objective of this assignment is to independently develop a classification model for each of the two provided datasets. You will experiment with three different algorithms (excluding transformers) and analyze their performance

Contains titles, abstracts, and topic labels of research articles. Your task is
to predict the topic based on the title and abstract. 

what algorithms i can use for the label based classification.

- multiclass classification algorithm
  - MLP
  - naive bayes
  - Logistic Regression utilising softmax
  - SVMs / decision trees

In [4]:
pip install openpyxl nltk

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     ------------------ ------------------- 20.5/41.5 kB 108.9 kB/s eta 0:00:01
     ---------------------------- --------- 30.7/41.5 kB 145.2 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 166.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 445.2 kB/s eta 0:00:04
    --------------------------------------- 0.0/1.5 MB 445.2 kB/s eta 0:00:04
    --------------------------------------- 0.0/1.5 MB 445.2 kB/s eta 0:00:04
   - -------------------------------------- 0.1/1.5 MB 262.6 kB/s eta 0:00:06
   - ------------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\pantm\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [41]:
import pandas as pd
df = pd.read_excel("Dataset-1.xlsx")
print(df)
df.info()

          ID                                              TITLE  \
0          1        Reconstructing Subject-Specific Effect Maps   
1          2                 Rotation Invariance Neural Network   
2          3  Spherical polyharmonics and Poisson kernels fo...   
3          4  A finite element approximation for the stochas...   
4          5  Comparative study of Discrete Wavelet Transfor...   
...      ...                                                ...   
20967  20968  Contemporary machine learning: a guide for pra...   
20968  20969  Uniform diamond coatings on WC-Co hard alloy c...   
20969  20970  Analysing Soccer Games with Clustering and Con...   
20970  20971  On the Efficient Simulation of the Left-Tail o...   
20971  20972   Why optional stopping is a problem for Bayesians   

                                                ABSTRACT  Computer Science  \
0        Predictive models allow subject-specific inf...                 1   
1        Rotation invariance and transl

In [9]:
## Data distribution
## which graphs or plots better suggests the data distributions? -> histogram, boxplots
# import matplotlib.pyplot as plt 
# plt.bar(X["Title"],y)
df.head(10)

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
5,6,On maximizing the fundamental frequency of the...,Let $\Omega \subset \mathbb{R}^n$ be a bound...,0,0,1,0,0,0
6,7,On the rotation period and shape of the hyperb...,We observed the newly discovered hyperbolic ...,0,1,0,0,0,0
7,8,Adverse effects of polymer coating on heat tra...,The ability of metallic nanoparticles to sup...,0,1,0,0,0,0
8,9,SPH calculations of Mars-scale collisions: the...,We model large-scale ($\approx$2000km) impac...,0,1,0,0,0,0
9,10,$\mathcal{R}_{0}$ fails to predict the outbrea...,Time varying susceptibility of host at indiv...,0,0,0,0,1,0


In [42]:
## train-test split

# X -> TITLE, ABSTRACT and Y -> labels(Phy, Maths, Stats, Quantitative Biology, Quant Finance)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay

df['text'] = df['TITLE'] + ' ' + df['ABSTRACT']
X = df['text']
y = df[['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']]


# feature engineering
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_tfidf = vectorizer.fit_transform(X)
X_train, X_test, y_train,y_test = train_test_split(X_tfidf,y,test_size=0.2,random_state=42)

# why have we chosen the MultinomialNB 
nb = MultinomialNB(alpha=0.1)
multi_target_nb = MultiOutputClassifier(nb)
multi_target_nb.fit(X_train,y_train)

y_pred = multi_target_nb.predict(X_test)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.77      0.87      0.82      1692
           1       0.93      0.81      0.87      1226
           2       0.82      0.76      0.79      1150
           3       0.73      0.79      0.76      1069
           4       0.67      0.18      0.28       122
           5       0.86      0.27      0.41        45

   micro avg       0.80      0.80      0.80      5304
   macro avg       0.80      0.61      0.65      5304
weighted avg       0.81      0.80      0.80      5304
 samples avg       0.80      0.82      0.79      5304



C:\Users\pantm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Model building and training

from sklearn.naive_bayes import GaussianNB
#TODO: based on my data distribution how do I decide which classifier to choose from naive_bayes scikit library
y

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,1,0,0,1,0,0
...,...,...,...,...,...,...
20967,1,1,0,0,0,0
20968,0,1,0,0,0,0
20969,1,0,0,0,0,0
20970,0,0,1,1,0,0


In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
def data_preprocessing(X,y):
    # utilize the logistic regression for the multi-label classification problem. 
    # preprocess the text data using techniques like TF-IDF or word embeddings to convert them into numerical features.
    vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    X_tfidf = vectorizer.fit_transform(X)
    X_train, X_test, y_train,y_test = train_test_split(X_tfidf,y,test_size=0.2,random_state=42)
    # which one to chose and when for example b/w the TFIDF one and the word embeddings
    base_model = LogisticRegression(multi_class='multinomial',solver='lbfgs',max_iter=1000,C=1.0)
    multi_model = MultiOutputClassifier(base_model)
    multi_model.fit(X_train,y_train)
    y_pred = multi_model.predict(X_test)
    print("Accuracy is: ",accuracy_score(y_test,y_pred))
    
data_preprocessing(X,y)


C:\Users\pantm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\pantm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\pantm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2k

Accuracy is:  0.6522050059594756


# SVM Approach

In [ ]:
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

X = df['TITLE'] + ' ' + df['ABSTRACT']
y = df[['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']]

vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_tfidf = vectorizer.fit_transform(X)
X_train, X_test, y_train,y_test = train_test_split(X_tfidf,y,test_size=0.2,random_state=42)

# what is the kernel in SVM 
svm = SVC(kernel='linear',probability=True)
multi_svm = MultiOutputClassifier(svm)
multi_svm.fit(X_train,y_train)
y_pred = multi_svm.predict(X_train)
accuracy = accuracy_score(y_pred,y_test)
print(" accuracy score is ", accuracy)


In [47]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 968333 stored elements and shape (16777, 5000)>
  Coords	Values
  (0, 3820)	0.057549140282289156
  (0, 1180)	0.034975370011717924
  (0, 2691)	0.054608614269348654
  (0, 2864)	0.03929349174465749
  (0, 533)	0.06990351224906194
  (0, 231)	0.044584790426710603
  (0, 3925)	0.03320578834359145
  (0, 4597)	0.048511194831599845
  (0, 4575)	0.052226884662400525
  (0, 772)	0.1197778819647435
  (0, 4622)	0.03828805449116095
  (0, 4013)	0.06507155526518951
  (0, 518)	0.06069618323610198
  (0, 4061)	0.06567256104409243
  (0, 4805)	0.04313413205367747
  (0, 4133)	0.046508041276487444
  (0, 1642)	0.07025751636434542
  (0, 3138)	0.05287711693222406
  (0, 4265)	0.1630848877448597
  (0, 4844)	0.06397713507924703
  (0, 2822)	0.06032517927336378
  (0, 3655)	0.356016629598416
  (0, 1092)	0.05863877897990759
  (0, 4927)	0.06648295291202731
  (0, 3728)	0.047469363612620954
  :	:
  (16776, 1850)	0.061654724205917034
  (16776, 162)	0.05943832236793

## Questions
- what is RBF kernel in svm : fits linear model in transformed space
- what is the penalty in logistic regression


## MLP Implementation for the classification task

### questions
  - how do the input size decided .

In [49]:


from sklearn.neural_network import MLPClassifier
#TODO: try to implement the following and compare the results.

'''
Best Parameters: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (5,), 'solver': 'sgd'}
Best Score: 0.9987261146496815
'''


mlp_classifier = MLPClassifier(hidden_layer_sizes=(5,10),activation='relu',solver='adam',alpha=0.0001)
mlp_classifier.fit(X_train,y_train)
y_pred = mlp_classifier.predict(X_test)

# Round predictions to 0 or 1
y_pred_binary = (y_pred > 0.5).astype(int)

print(classification_report(y_test, y_pred_binary, target_names=[
    'Computer Science', 'Physics', 'Mathematics', 'Statistics', 
    'Quantitative Biology', 'Quantitative Finance'
]))

# Calculate overall accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Overall Accuracy: {accuracy:.4f}")


                      precision    recall  f1-score   support

    Computer Science       0.74      0.74      0.74      1692
             Physics       0.77      0.84      0.81      1226
         Mathematics       0.72      0.75      0.73      1150
          Statistics       0.63      0.64      0.63      1069
Quantitative Biology       0.46      0.26      0.33       122
Quantitative Finance       0.73      0.36      0.48        45

           micro avg       0.72      0.73      0.72      5304
           macro avg       0.67      0.60      0.62      5304
        weighted avg       0.71      0.73      0.72      5304
         samples avg       0.74      0.76      0.73      5304

Overall Accuracy: 0.5275


C:\Users\pantm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\pantm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [51]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
# Define the MLP model
mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64, 32),  # Three hidden layers
    activation='relu',                 # ReLU activation function
    solver='adam',                     # Adam optimizer
    alpha=0.0001,                      # L2 regularization parameter
    batch_size='auto',                 # Automatic batch size
    learning_rate='adaptive',          # Adaptive learning rate
    max_iter=1000,                     # Maximum number of iterations
    random_state=42,                   # For reproducibility
    verbose=True                       # Print progress
)

# Train the model
mlp.fit(X_train, y_train)
# Make predictions
y_pred = mlp.predict(X_test)

# Round predictions to 0 or 1
y_pred_binary = (y_pred > 0.5).astype(int)

# Print performance metrics
print("Classification Report:")
print(classification_report(y_test, y_pred_binary, target_names=[
    'Computer Science', 'Physics', 'Mathematics', 'Statistics', 
    'Quantitative Biology', 'Quantitative Finance'
]))

# Calculate overall accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Overall Accuracy: {accuracy:.4f}")


Iteration 1, loss = 2.68791136
Iteration 2, loss = 1.33266176
Iteration 3, loss = 0.96995388
Iteration 4, loss = 0.82904822
Iteration 5, loss = 0.71221854
Iteration 6, loss = 0.60149460
Iteration 7, loss = 0.48855522
Iteration 8, loss = 0.37332579
Iteration 9, loss = 0.26664911
Iteration 10, loss = 0.17548167
Iteration 11, loss = 0.10798356
Iteration 12, loss = 0.06522094
Iteration 13, loss = 0.03973699
Iteration 14, loss = 0.02578171
Iteration 15, loss = 0.01789852
Iteration 16, loss = 0.01312397
Iteration 17, loss = 0.01018717
Iteration 18, loss = 0.00817570
Iteration 19, loss = 0.00670836
Iteration 20, loss = 0.00563122
Iteration 21, loss = 0.00478494
Iteration 22, loss = 0.00412408
Iteration 23, loss = 0.00362020
Iteration 24, loss = 0.00323461
Iteration 25, loss = 0.00292752
Iteration 26, loss = 0.00268365
Iteration 27, loss = 0.00248500
Iteration 28, loss = 0.00231987
Iteration 29, loss = 0.00217825
Iteration 30, loss = 0.00205868
Iteration 31, loss = 0.00195466
Iteration 32, los

C:\Users\pantm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Question2: Contains text from certain domains and you have to predict the domain.

- i shall be using dataprocessing
- tf-idf followed by the naive bayes classification and 
- word embeddings using the 

In [3]:
import pandas as pd
df_a2d2 = pd.read_excel("A2D2.xlsx")
print(df_a2d2)
df_a2d2.info()

        ID                                            Content         Domain
0        1  engali Binodiini Ekti Natir Putul Chaalchitro ...  Entertainment
1        2   ChiefsAholic A Wolf In Chiefs Clothing articl...  Entertainment
2        3  Kabandha Your Rating Write a review Optional C...  Entertainment
3        4  In Bruges 2008 R 1h 47m IMDb RATING 79 10 474K...  Entertainment
4        5  Men in Black 2012 PG13 1h 46m IMDb RATING 68 1...  Entertainment
...    ...                                                ...            ...
3922  3923   Kerala with its Munnar Wayanad Kochi Alleppey...        Tourism
3923  3924   Netaji Subhash Chandra Bose Airport CCU Kolka...        Tourism
3924  3925  Spanning over 800 square kilometres in the Alw...        Tourism
3925  3926  Located in the Eastern part of India West Beng...        Tourism
3926  3927   Maharaja Bir Bikram Airport IXA Agartala Agar...        Tourism

[3927 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3

In [1]:
# Perform data preprocessing stemming and lemmitization
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
# Ensure required resources are downloaded
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt_tab', quiet=True)

True

In [6]:
def preprocess_text(text):
    text = text.lower()
    #text = re.sub(r'^a-zA-Z\s','',text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text) # internally how it works show with an example.
    #remove stopwords
    tokens=[word for word in tokens if word not in stopwords.words("english")]
    # lemmatizer
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

df_a2d2['processed_text'] = df_a2d2['Content'].apply(preprocess_text)
df_a2d2.head()


,ID,Content,Domain,processed_text
0,1,engali Binodiini Ekti Natir Putul Chaalchitro ...,Entertainment,engali binodiini ekti natir putul chaalchitro ...
1,2,ChiefsAholic A Wolf In Chiefs Clothing articl...,Entertainment,chiefsaholic wolf chief clothing articleshowcm...
2,3,Kabandha Your Rating Write a review Optional C...,Entertainment,kabandha rating write review optional characte...
3,4,In Bruges 2008 R 1h 47m IMDb RATING 79 10 474K...,Entertainment,bruges r h imdb rating k rating rate popularit...
4,5,Men in Black 2012 PG13 1h 46m IMDb RATING 68 1...,Entertainment,men black pg h imdb rating k rating rate popul...


## tf-idf based modeld

In [26]:
df_a2d2['Domain'].unique()

array(['Entertainment', 'Healthcare', 'Sports', 'Technology', 'Tourism'],
      dtype=object)

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = df_a2d2['processed_text']
y = df_a2d2['Domain']
X_tfidf = vectorizer.fit_transform(X)
X_train, X_test, y_train,y_test = train_test_split(X_tfidf,y,test_size=0.2,random_state=42)

## Naive Bayes

In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report

multinomial_nb = MultinomialNB()
multinomial_nb.fit(X_train,y_train)
y_pred = multinomial_nb.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9987277353689568

Classification Report:
                precision    recall  f1-score   support

Entertainment       0.99      1.00      1.00       146
   Healthcare       1.00      1.00      1.00       193
       Sports       1.00      1.00      1.00        73
   Technology       1.00      1.00      1.00       260
      Tourism       1.00      1.00      1.00       114

     accuracy                           1.00       786
    macro avg       1.00      1.00      1.00       786
 weighted avg       1.00      1.00      1.00       786



## SVM for question

In [54]:
from sklearn.svm import SVC 
X_train, X_test, y_train, y_test = train_test_split(
    df_a2d2['processed_text'], df_a2d2['Domain'], test_size=0.2, random_state=42
)

# Step 3: Apply TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

svm_classifier = SVC(kernel='linear',random_state=42)
svm_classifier.fit(X_train_tfidf,y_train)
y_pred  = svm_classifier.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 1.0

Classification Report:
                precision    recall  f1-score   support

Entertainment       1.00      1.00      1.00       146
   Healthcare       1.00      1.00      1.00       193
       Sports       1.00      1.00      1.00        73
   Technology       1.00      1.00      1.00       260
      Tourism       1.00      1.00      1.00       114

     accuracy                           1.00       786
    macro avg       1.00      1.00      1.00       786
 weighted avg       1.00      1.00      1.00       786



## apply MLP algorithm
- in my case let us say i have some 3XXX rows and 1 column, what shall be my ideal input, hidden and output layers.
  - outputlayer: 5
  - hidden layers:  
  - input layers: 
  - what shall be the size a single, 2 hidden or 3 hidden layers architecture
- how do we check for the overfitting or the underfitting of my data

In [ ]:
from sklearn.neural_network import  MLPClassifier

mlp_classifier = MLPClassifier(
    hidden_layer_sizes=(3,),#(128, 64, 32),  # Three hidden layers
    activation='tanh',                 # ReLU activation function
    solver='sgd',                     # Adam optimizer
    alpha=0.0001,                      # L2 regularization parameter
    batch_size='auto',                 # Automatic batch size
    learning_rate='adaptive',          # Adaptive learning rate
    max_iter=1000,                     # Maximum number of iterations
    random_state=42,                   # For reproducibility
    verbose=True                       # Print progress
)

mlp_classifier.fit(X_train,y_train)
y_pred = mlp_classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Iteration 1, loss = 1.48580668
Iteration 2, loss = 1.42508813
Iteration 3, loss = 1.36335755
Iteration 4, loss = 1.30025341
Iteration 5, loss = 1.23852523
Iteration 6, loss = 1.17945750
Iteration 7, loss = 1.12293472
Iteration 8, loss = 1.06916004
Iteration 9, loss = 1.01801782
Iteration 10, loss = 0.96865724
Iteration 11, loss = 0.92087251
Iteration 12, loss = 0.87401414
Iteration 13, loss = 0.82838432
Iteration 14, loss = 0.78417650
Iteration 15, loss = 0.74137804
Iteration 16, loss = 0.70066267
Iteration 17, loss = 0.66201728
Iteration 18, loss = 0.62554298
Iteration 19, loss = 0.59168109
Iteration 20, loss = 0.56010180
Iteration 21, loss = 0.53087872
Iteration 22, loss = 0.50406639
Iteration 23, loss = 0.47946030
Iteration 24, loss = 0.45683488
Iteration 25, loss = 0.43624696
Iteration 26, loss = 0.41740905
Iteration 27, loss = 0.40031668
Iteration 28, loss = 0.38487584
Iteration 29, loss = 0.37069409
Iteration 30, loss = 0.35787277
Iteration 31, loss = 0.34622807
Iteration 32, los

In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification

# Define the model
model = MLPClassifier(max_iter=1000, random_state=42)

# Define the hyperparameter grid
param_grid = {
    'hidden_layer_sizes': [(5,), (3,), (25, 5)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001],
}

# Perform Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train,y_train)

# Print the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

C:\Users\pantm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\pantm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\pantm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\pant

Best Parameters: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (5,), 'solver': 'sgd'}
Best Score: 0.9987261146496815


In [18]:
# let us try this without the lemmitization step and check with 
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = df_a2d2['Content']
y = df_a2d2['Domain']
X_tfidf = vectorizer.fit_transform(X)
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [19]:
print(X_train)
multinomial_nb = MultinomialNB()
multinomial_nb.fit(X_train,y_train)
y_pred = multinomial_nb.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


2758    Over the last few years haptic technology has ...
3696    Abu Dhabi Airport 5h 15m One way 10891 Onwards...
3864    A millennia old tradition of storytelling Maha...
2510    The Redmi Note 11 Pro 5G is priced starting at...
2172    Amphenol Commercial USB Type C Connector Syste...
                              ...                        
1130    ncird divis offic ncird cdc skip directli site...
1294    Considerations The symptoms of an ear infectio...
860     us govern releas first nation one health plan ...
3507    Famous for watersports Popular for water sport...
3174    Blockchain AI research lab YeagerAI has announ...
Name: Content, Length: 3141, dtype: object


ValueError: could not convert string to float: 'Over the last few years haptic technology has undergone significant advancements across a wide range of industries Haptic devices are designed to create a sense of touch through vibration force or motion in environments where the opportunities for audio and visual feedback are limited From consumer electronics to automotive interfaces haptic feedback has become an integral feature that can enhance user experiences and create nextgen humanmachine interfaces HMI There has been a dynamic transition from conventional solutions to piezobased haptic technologies which provide speedy response times over highfrequency bandwidths in ultracompact form factors These devices have become essential in todays automobiles where limited space and power efficiencies dictate how haptic feedback can be employed on steering wheels dashboards and displays Figure 1 In todays modern cars haptic technologies play a key role on steering wheels dashboards and displ